In [1]:
from theano.sandbox import cuda
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
Using Theano backend.


In [2]:
from keras.datasets import imdb
idx = imdb.get_word_index()

1654784/1641221 [==============================] - 0s 0us/step


In [8]:
print(len(idx))
for k,v in idx.items():
    print("k = %s, v = %s"%(k, v))
    break


88584
k = fawn, v = 34701


In [16]:
idx2word = {i:w for w,i in idx.items()}

In [18]:
idx2word[34701]

u'fawn'

In [19]:
path = get_file('imdb_full.pkl',
                origin='https://s3.amazonaws.com/text-datasets/imdb_full.pkl',
                md5_hash='d091312047c43cf9e4e38fef92437263')
f = open(path, 'rb')
(x_train, labels_train), (x_test, labels_test) = pickle.load(f)

65568768/65552540 [==============================] - 1s 0us/step


In [58]:
print(' '.join([idx2word[i] for i in x_train[0]]))
print(len(x_train[1]))

bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell high's satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers' pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector i'm here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isn't
433


In [43]:
', '.join(map(str, x_train[0]))

'23022, 309, 6, 3, 1069, 209, 9, 2175, 30, 1, 169, 55, 14, 46, 82, 5869, 41, 393, 110, 138, 14, 5359, 58, 4477, 150, 8, 1, 5032, 5948, 482, 69, 5, 261, 12, 23022, 73935, 2003, 6, 73, 2436, 5, 632, 71, 6, 5359, 1, 25279, 5, 2004, 10471, 1, 5941, 1534, 34, 67, 64, 205, 140, 65, 1232, 63526, 21145, 1, 49265, 4, 1, 223, 901, 29, 3024, 69, 4, 1, 5863, 10, 694, 2, 65, 1534, 51, 10, 216, 1, 387, 8, 60, 3, 1472, 3724, 802, 5, 3521, 177, 1, 393, 10, 1238, 14030, 30, 309, 3, 353, 344, 2989, 143, 130, 5, 7804, 28, 4, 126, 5359, 1472, 2375, 5, 23022, 309, 10, 532, 12, 108, 1470, 4, 58, 556, 101, 12, 23022, 309, 6, 227, 4187, 48, 3, 2237, 12, 9, 215'

In [112]:
vocal_size = 5000
trn1 = [np.array([j if j < vocal_size - 1 else vocal_size - 1 for j in i]) for i in  x_train]
test1 = [np.array([j if j < vocal_size - 1 else vocal_size - 1 for j in i]) for i in  x_test]
print(trn1[0])

[4999  309    6    3 1069  209    9 2175   30    1  169   55   14   46   82 4999   41  393  110  138
   14 4999   58 4477  150    8    1 4999 4999  482   69    5  261   12 4999 4999 2003    6   73 2436
    5  632   71    6 4999    1 4999    5 2004 4999    1 4999 1534   34   67   64  205  140   65 1232
 4999 4999    1 4999    4    1  223  901   29 3024   69    4    1 4999   10  694    2   65 1534   51
   10  216    1  387    8   60    3 1472 3724  802    5 3521  177    1  393   10 1238 4999   30  309
    3  353  344 2989  143  130    5 4999   28    4  126 4999 1472 2375    5 4999  309   10  532   12
  108 1470    4   58  556  101   12 4999  309    6  227 4187   48    3 2237   12    9  215]


In [114]:
lens = np.array(map(len, trn1))
(lens.max(), lens.min(), lens.mean())

(2493, 10, 237.71364)

In [123]:
seq_len = 500
trn = [np.lib.pad(t, (0, seq_len - len(t)), 'constant', constant_values=(0)) if len(t) < seq_len else t[:seq_len] for t in trn1]
test = [np.lib.pad(t, (0, seq_len - len(t)), 'constant', constant_values=(0)) if len(t) < seq_len else t[:seq_len] for t in test1]
trn = np.array(trn)
test = np.array(test)

In [126]:
print(trn.shape)
print(test.shape)

(25000, 500)
(25000, 500)


In [118]:
lens = np.array(map(len, trn))
print(lens.max(), lens.min(), lens.mean())
a = [1, 2, 3, 4, 5]
np.lib.pad(a, (0,3), 'constant', constant_values=(0))

500 500 500.0


array([1, 2, 3, 4, 5, 0, 0, 0])

In [128]:
model = Sequential([
        Embedding(vocal_size, 32, input_length=seq_len),
        Flatten(),
        Dense(100, activation='relu'),
        Dropout(0.7),
        Dense(1, activation='sigmoid')
    ])

In [129]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 16000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               1600100   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 1,760,201
Trainable params: 1,760,201
Non-trainable params: 0
_________________________________________________________________


In [131]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 4s 159us/step - loss: 0.0979 - acc: 0.9674 - val_loss: 0.4342 - val_acc: 0.8471
Epoch 2/2
25000/25000 [==============================] - 4s 153us/step - loss: 0.0347 - acc: 0.9904 - val_loss: 0.4973 - val_acc: 0.8526


In [132]:
conv1 = Sequential([
        Embedding(vocal_size, 32, input_length=seq_len, dropout=0.2),
        Dropout(0.2),
        Convolution1D(64, 5, border_mode='same', activation='relu'),
        Dropout(0.2),
        MaxPooling1D(),
        Flatten(),
        Dense(100, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  from ipykernel import kernelapp as app
/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 5, padding="same", activation="relu")`


In [133]:
conv1.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
conv1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
dropout_2 (Dropout)          (None, 500, 32)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 500, 64)           10304     
_________________________________________________________________
dropout_3 (Dropout)          (None, 500, 64)           0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 250, 64)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 16000)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               1600100   
__________

In [134]:
conv1.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 16s 639us/step - loss: 0.4678 - acc: 0.7350 - val_loss: 0.2916 - val_acc: 0.8796
Epoch 2/2
25000/25000 [==============================] - 16s 636us/step - loss: 0.2298 - acc: 0.9097 - val_loss: 0.2699 - val_acc: 0.8864


In [135]:
conv1.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=5, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 16s 640us/step - loss: 0.1786 - acc: 0.9304 - val_loss: 0.2822 - val_acc: 0.8840
Epoch 2/5
25000/25000 [==============================] - 16s 634us/step - loss: 0.1453 - acc: 0.9467 - val_loss: 0.3112 - val_acc: 0.8754
Epoch 3/5
25000/25000 [==============================] - 16s 634us/step - loss: 0.1116 - acc: 0.9606 - val_loss: 0.3470 - val_acc: 0.8717
Epoch 4/5
25000/25000 [==============================] - 16s 634us/step - loss: 0.0866 - acc: 0.9698 - val_loss: 0.3896 - val_acc: 0.8718
Epoch 5/5
25000/25000 [==============================] - 16s 634us/step - loss: 0.0596 - acc: 0.9810 - val_loss: 0.4478 - val_acc: 0.8714
